Obligatory import for the notebook sake.

In [38]:
import os
import lsst.afw.geom as afwGeom
import lsst.afw.table as afwTable
import pickle

import numpy as np

import lsst.daf.persistence as dafPersist
import re
import astropy.coordinates as coord
import astropy.units as u

import matplotlib.pyplot as plt
from astropy.table import Table

File containing the Mjd dictionary. Directory definition (depending wich set of data is going to be used)

In [39]:
def source_distance(src1, src2):
    ra, dec = src1['ra'], src1['dec']
    ra2, dec2 = src2['ra'], src2['dec']
            
    return np.sqrt((float(ra)-float(ra2))**2+(float(dec)-float(dec2))**2)/3.14159*180*3600


def threshold_light_curves(light_curves, threshold):
    t_light_curves = [lc for lc in light_curves if len(lc) >= threshold]
    return t_light_curves

def build_light_curve_from_snls_file(data, c):

    bandpasses = ['r']


    lightcurve = {}
    lightcurve['bandpass'] = []
    lightcurve['mjd'] = []
    lightcurve['ra'] = []
    lightcurve['dec'] = []
    lightcurve['flux'] = []
    lightcurve['flux_error'] = []
    lightcurve['zp'] = []
    lightcurve['zpsys'] = []


    for mjd, flux, error in data:

        #print 'yep',visit
        lightcurve['bandpass'].append(str('sdss' + bandpasses[0]))
        lightcurve['mjd'].append(float(mjd))
        lightcurve['ra'].append(c.ra.radian)
        lightcurve['dec'].append(c.dec.radian)
        lightcurve['flux'].append(float(flux))
        lightcurve['flux_error'].append(float(error))
        #lightcurve['flux'].append(src['base_CircularApertureFlux_12_0_flux'])
        #lightcurve['flux_error'].append(src['base_CircularApertureFlux_12_0_fluxSigma'])
        lightcurve['zp'].append(25.0)
        lightcurve['zpsys'].append('ab')

    lc = Table(data=lightcurve)
    return lc

def build_lightcurve(source_list):
    """
    Assemble a light curve data table from available files.
    """

    bandpasses = ['r']


    lightcurve = {}
    lightcurve['classification'] = []
    lightcurve['bandpass'] = []
    lightcurve['mjd'] = []
    lightcurve['ra'] = []
    lightcurve['dec'] = []
    lightcurve['flux'] = []
    lightcurve['flux_error'] = []
    lightcurve['zp'] = []
    lightcurve['zpsys'] = []


    for visit, src in source_list:

        #print 'yep',visit
        lightcurve['classification'].append(src['classification_dipole'])
        lightcurve['bandpass'].append(str('sdss' + bandpasses[0]))
        lightcurve['mjd'].append(mjds[str(visit)])
        lightcurve['ra'].append(src['coord_ra'])
        lightcurve['dec'].append(src['coord_dec'])
        lightcurve['flux'].append(src['base_CircularApertureFlux_4_5_flux'])
        lightcurve['flux_error'].append(src['base_CircularApertureFlux_4_5_fluxSigma'])
        #lightcurve['flux'].append(src['base_CircularApertureFlux_12_0_flux'])
        #lightcurve['flux_error'].append(src['base_CircularApertureFlux_12_0_fluxSigma'])
        lightcurve['zp'].append(25.0)
        lightcurve['zpsys'].append('ab')
    lightcurve = Table(data=lightcurve)
    return lightcurve


def mean_lc_flux(lc2):

    mjds = []
    m_fluxes = []
    m_error = []
    s_error = []
    i = 0
    while i < len(lc2['mjd']):
        mjd = lc2['mjd'][i]
        j = i+1
        mjds.append(int(mjd))
        m_fluxes.append(lc2['flux'][i]*lc2['flux_error'][i])
        m_error.append(lc2['flux_error'][i])
        s_error.append(lc2['flux_error'][i]**2)
        total = 1.
       
        while j<len(lc2['mjd']) and int(lc2['mjd'][j]) == int(mjd):

            m_fluxes[-1]+= lc2['flux'][j]*lc2['flux_error'][j]
            

                
            m_error[-1] += lc2['flux_error'][j]
            s_error[-1] += (lc2['flux_error'][j]**2)
            #mjds[-1] += int(lc2['mjd'][j])
            total+=1
            j+=1
            
       
        m_fluxes[-1] = np.divide(m_fluxes[-1], m_error[-1] )
        m_error[-1] = np.divide(np.sqrt(s_error[-1]), total)
        i = j

    return mjds, m_fluxes, m_error


Light curves from SNLS are read.

Creating the butler instance from the main data directory.

In [49]:

DATADIR="/renoir_data_02/jpreyes/lsst_data/CFHTLS_master/"

butler = dafPersist.Butler(DATADIR+"/output")
visits = butler.queryMetadata("deepDiff_differenceExp", format=['visit'],dataId={'filter':'r'})
visits = np.array(visits)
mask = (visits >= 836493) & (visits <= 860150)
visits =  visits[mask]
print visits

[836493 836494 836495 836496 836497 836498 837002 837003 837004 837005
 837006 837007 837008 838853 838854 838855 838857 838864 838865 838866
 838867 838868 839308 839309 839310 839311 839312 844210 844211 844212
 844213 844214 844230 844489 844490 844491 844492 844493 844494 844495
 844496 844497 844498 844514 844515 844516 844517 844518 844845 844846
 844847 844848 844849 845345 845346 845347 845348 845349 849373 849374
 849375 849376 849377 849685 849686 849687 849688 849689 850177 850178
 850179 850180 850181 850586 850587 850588 850589 850590 851057 851058
 851059 851060 851061 852890 852891 852892 852893 852894 853232 853233
 853234 853235 853236 853539 853540 853541 853542 853543 853727 853728
 853729 853730 853731 858537 858538 858539 858540 858541 859608 859609
 859610 859611 859612 860146 860147 860148 860149 860150]


In [45]:
directory = "catalogs/4sigma/"
d_directory = "catalogs/4sigma_i/"

files = os.listdir(directory)

y, n = 0,0

for f in files:
    visit = int(f.split("-")[0])
    ccd = int(f.split("-")[-1].split(".")[0])
    if butler.datasetExists("deepDiff_differenceExp", {'visit': visit , 'filter':"i" , 'ccd':ccd}):
        y+=1
    else:
        n+=1
        
print y,n

0 0


In [56]:
directory = "catalogs/4sigma/"
d_directory = "catalogs/4sigma_i/"
files = os.listdir(directory)
for f in files:
    visit = int(f.split("-")[0])
    if int(visit) not in visits:
       
        os.rename(directory+f, d_directory+f)

In [53]:
print files

[]
